In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from IPython.core.display import HTML
display(HTML("<style>.p-Widget.jp-OutputPrompt.jp-OutputArea-prompt:empty {padding: 0; border: 0;}</style>"));

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import os, sys, pickle
sys.path.append('..')

In [4]:
np.random.seed(123)

# Neural Statistics By Cell

In [5]:
response_dir = 'response_arrays/'
cell_metadata = pd.read_csv(os.path.join(response_dir, 'cell_metadata.csv')
cell_responses = pickle.load(open(os.path.join(response_dir, 'response_average_bycell.pkl', 'rb')))
cell_responses_bytrial = pickle.load(open(os.path.join(response_dir, 'responses_bytrial_bycell.pkl', 'rb')))

In [6]:
from toolbox.reliability import split_half

output_file = 'reliabilities_bycell.csv'
if os.path.exists(output_file):
    neural_reliabilities = pd.read_csv(output_file)
    
if not os.path.exists(output_file):
    neural_reliabilities_dictlist = []
    for cell_id in tqdm(cell_metadata['cell_specimen_id']):
        reference_array = cell_responses[cell_id]
        response_array = cell_responses_bytrial[cell_id]
        response_by_image_array = np.zeros(shape=(response_array.shape[0] // 50, 50))
        for image in range(response_array.shape[0] // 50):
            response_by_image_array[image] = response_array[50*image:50*(image+1)]
        image_by_response_array = np.swapaxes(response_by_image_array, 0, 1)    
        response_reliability = split_half(image_by_response_array)[0]
        neural_reliabilities_dictlist.append({'cell_specimen_id': cell_id, 'splithalf_r': response_reliability})
        if not np.allclose(reference_array, response_by_image_array.mean(axis=1)):
            print(cell_id)

    neural_reliabilities = pd.DataFrame(neural_reliabilities_dictlist)
    neural_reliabilities.to_csv(output_file, index = None)

In [7]:
neural_reliabilities.query('splithalf_r > 0.75')

,cell_specimen_id,splithalf_r
1,662220119,0.820679
2,662220063,0.879785
5,662220048,0.966658
11,662220001,0.918633
12,662220053,0.800464
...,...,...
39242,662225725,0.775695
39243,662223605,0.812394
39247,662222176,0.948436
39262,517504625,0.871873


In [9]:
cell_data_plus = cell_metadata.merge(neural_reliabilities, on=['cell_specimen_id'])

output_file = 'cell_data_plus.csv'
if not os.path.exists(output_file):
    cell_data_plus.to_csv(output_file, index = None)

# Neural Statistics By Site

In [ ]:
#Before anything else, we ensure our arrays are properly shaped.

In [ ]:
excitatory = ['Emx1', 'Slc17a7', 'Cux2', 'Rorb', 'Scnn1a', 'Nr5a1', 'Rbp4', 'Fezf2', 'Tlx3', 'Ntsr1']
inhibitory = ['Sst', 'Vip', 'Pvalb']

In [ ]:
with open('responses_bytrial_bysite.pkl', 'rb') as f:
    brain_responses = pickle.load(f)

In [ ]:
visual_areas = list(brain_responses.keys())
layers = ['layer23', 'layer4', 'layer5', 'layer6']

In [ ]:
brain_response_summary = {}
for area in visual_areas:
    brain_response_summary[area] = {}
    for layer in layers:
        brain_response_summary[area][layer] = []
        for cre_line in list(brain_responses[area].keys()):
            if cre_line not in inhibitory:
                if layer in brain_responses[area][cre_line].keys():
                    #brain_response_summary[area][layer].append(brain_responses[area][cre_line][layer].transpose())
                    brain_response_summary[area][layer].append(brain_responses[area][cre_line][layer])
        if len(brain_response_summary[area][layer]) == 0:
            brain_response_summary[area].pop(layer)

In [ ]:
brain_responses_bytrial = {}
for area in visual_areas:
    brain_responses_bytrial[area] = {}
    for layer in layers:
        if layer in brain_response_summary[area].keys():
            brain_responses_bytrial[area][layer] = np.hstack(brain_response_summary[area][layer])
            #print(area, layer, os.linesep, brain_responses_bytrial[area][layer].shape)

In [ ]:
with open(os.path.join(response_dir, 'response_average_bysite.pkl', 'rb')) as file:
    response_dict = pickle.load(file)

brain_responses_bytrial_aggregate = []
for image_index in range(5950 // 50):
    brain_responses_bytrial_aggregate.append(np.mean(brain_responses_bytrial['VISl']['layer23'][50*image_index:50*(image_index+1), :], axis = 0))
brain_responses_bytrial_aggregate = np.vstack(brain_responses_bytrial_aggregate)

brain_responses_byimage = {}
for area in response_dict.keys():
    resp_exc = {'layer23':[], 'layer4':[], 'layer5':[], 'layer6':[]}
    resp_inh = {'layer23':[], 'layer4':[], 'layer5':[], 'layer6':[]}
    brain_responses_byimage[area] = {}
    for cre in response_dict[area].keys():
        for layer in response_dict[area][cre].keys():
            #print(area, cre, layer, os.linesep, response_dict[area][cre][layer].shape)

            response = response_dict[area][cre][layer]

            if cre in excitatory:
                resp_exc[layer] += [response]
            if cre in inhibitory:
                resp_inh[layer] += [response]
                
    for layer in resp_exc:
        
        if len(resp_exc[layer])>0:
            resp_e = np.hstack(resp_exc[layer])
            brain_responses_byimage[area][layer] = resp_e
            #print(area, layer, os.linesep, resp_e.shape)
            
print('Trial arrays properly computed?', np.array_equal(brain_responses_bytrial_aggregate, brain_responses_byimage['VISl']['layer23']))

In [ ]:
brain_responses_bytrial['VISl']['layer23'].shape

In [ ]:
brain_responses_byimage['VISl']['layer23'].shape

## Reliability Calculations

In [ ]:
from toolbox.reliability import split_half

output_file = 'reliabilities_bysite.pkl'
if os.path.exists(output_file):
    with open(output_file, 'rb') as file:
        neural_reliabilities = pickle.load(file)
    
if not os.path.exists(output_file):
    neural_reliabilities = {}
    for area in tqdm(visual_areas):
        neural_reliabilities[area] = {}
        layers = brain_responses_bytrial[area].keys()
        for layer in tqdm(layers, leave = False):
            response_array = brain_responses_bytrial[area][layer]
            neural_array = np.zeros(shape=(response_array.shape[1], response_array.shape[0] // 50, 50))
            for neuron in range(response_array.shape[1]):
                for image in range(response_array.shape[0] // 50):
                    neural_array[neuron][image] = response_array[50*image:50*(image+1), neuron]

            neural_array = np.swapaxes(neural_array, 1, 2)
            neural_reliability = np.zeros((neural_array.shape[0], 2))
            for neuron in tqdm(range(neural_array.shape[0]), desc = ' '.join([area, layer]), leave = False):
                neural_reliability[neuron] = split_half(neural_array[neuron,:,:])

            neural_reliabilities[area][layer] = neural_reliability
            
    neural_reliabilities_dictlist = []
    for area in list(neural_reliabilities.keys()):
        for layer in list(neural_reliabilities[area].keys()):
            for neural_index, neuron in enumerate(neural_reliabilities[area][layer]):
                neural_reliabilities_dictlist.append({'area' : area, 'layer': layer, 'neural_site': area+'-'+layer,
                                                      'neuron': neural_index, 'splithalf_r': neuron[0], 'splithalf_sem': neuron[0]})
    
    with open(output_file, 'wb') as file:
        pickle.dump(neural_reliabilities, file)

In [ ]:
neural_reliabilities_dictlist = []
for area in list(neural_reliabilities.keys()):
    for layer in list(neural_reliabilities[area].keys()):
        for neural_index, neuron in enumerate(neural_reliabilities[area][layer]):
            neural_reliabilities_dictlist.append({'area' : area, 'layer': layer, 'neural_site': area+'-'+layer,
                                                  'neuron': neural_index, 'splithalf_r': neuron[0], 'splithalf_sem': neuron[0]})

In [ ]:
reliability_bysite_dictlist = []
for area in visual_areas:
    for layer in neural_reliabilities[area].keys():
        reliability = np.mean(neural_reliabilities[area][layer], axis = 0)
        reliability_bysite_dictlist.append({'area': area, 'layer': layer, 'neural_site': area+'-'+layer,
                                            'number_of_neurons': neural_reliabilities[area][layer].shape[0],
                                            'reliability': reliability[0], 'sem': reliability[1]})

reliability_bysite = pd.DataFrame(reliability_bysite_dictlist)

In [ ]:
plt.figure(figsize=(15,10))
p = sns.regplot(x='number_of_neurons', y='reliability', data=reliability_bysite);

for point in range(0, reliability_bysite.shape[0]):
     p.text(reliability_bysite['number_of_neurons'][point]+0.01, reliability_bysite['reliability'][point], 
     reliability_bysite['neural_site'][point], horizontalalignment='left', 
     size='medium', color='black', weight='semibold')

In [ ]:
total_neurons = 0
for area in visual_areas:
    for layer in neural_reliabilities[area].keys():
        total_neurons += neural_reliabilities[area][layer].shape[0]
        print(area, layer, ':', neural_reliabilities[area][layer].shape[0])
print('Total neurons: ', total_neurons)

In [ ]:
reliability_count_list = []
for cutoff in tqdm(np.linspace(0,0.99,100)):
    reliable_neurons = {}
    for area in visual_areas:
        reliable_neurons[area] = {}
        for layer in neural_reliabilities[area].keys():
            reliable_neurons[area][layer] = []
            reliable_neurons_in_layer = 0
            for neural_index, neuron in enumerate(neural_reliabilities[area][layer]):
                if neuron[0] >= cutoff:
                    reliable_neurons[area][layer].append(neural_index)
            reliable_neurons[area][layer] = np.array(reliable_neurons[area][layer])
            reliability_count_list.append({'area': area, 'layer': layer, 'neural_site': area+'-'+layer,
                                           'cutoff': cutoff, 'count': reliable_neurons[area][layer].shape[0]})
            
reliability_counts = pd.DataFrame(reliability_count_list)

### Reliability Selection

In [ ]:
p = sns.relplot(x='cutoff', y='count', hue='neural_site', data = reliability_counts)

In [ ]:
sns.scatterplot(x='cutoff', y='count', data = reliability_counts.groupby('cutoff')['count'].sum().reset_index());

In [ ]:
from scipy.stats import pearsonr

def spearman_brown(pearson_r):
    return 2.0 * pearson_r / (1.0 + pearson_r)

output_file = 'reliability_selection_bysite.csv'
if os.path.exists(output_file):
    reliabilities = pd.read_csv(output_file)
    
if not (os.path.exists(output_file)):
    reliabilities_list = []
    for cutoff in tqdm(np.linspace(0.0,0.99,100)):
        for area in tqdm(visual_areas, leave = False):
            for layer in tqdm(list(brain_responses_bytrial[area].keys()), leave = False):
                reliable_neurons = [i for (i, el) in enumerate(neural_reliabilities[area][layer]) if el[0] >= cutoff]
                response_array = brain_responses_bytrial[area][layer][:, reliable_neurons]
                neural_array = np.zeros(shape=(response_array.shape[1], response_array.shape[0] // 50, 50))
                for neuron in range(response_array.shape[1]):
                    for image in range(response_array.shape[0] // 50):
                        neural_array[neuron][image] = response_array[50*image:50*(image+1), neuron]
                        
                iter_dict1 = {'cutoff': cutoff, 'area': area, 'layer': layer, 
                             'neural_site': area+'-'+layer, 'neuron_count': len(reliable_neurons)}
                trial_indices = np.array(range(50))
                neural_array = np.swapaxes(neural_array,0,2)
                if neural_array.shape[2] > 3:
                    for i in range(10):
                        np.random.shuffle(trial_indices)
                        similarity1 = np.corrcoef(np.mean(neural_array[trial_indices[:25], :, : ], axis=0))
                        similarity2 = np.corrcoef(np.mean(neural_array[trial_indices[25:], :, : ], axis=0))
                        sim1_triu = similarity1[np.triu_indices(similarity1.shape[0], k=1)].flatten()
                        sim2_triu = similarity2[np.triu_indices(similarity2.shape[0], k=1)].flatten()
                        rdm_pearson = pearsonr(np.nan_to_num(sim1_triu), np.nan_to_num(sim2_triu))[0]
                        iter_dict2 = {'iteration': i, 'instance': 'rdm', 
                                      'splithalf_r': rdm_pearson, 'spearman_brown': spearman_brown(rdm_pearson)}
                        reliabilities_list.append({**iter_dict1, **iter_dict2})

                        image_splitcorrs = np.zeros(119)
                        for image in range(neural_array.shape[1]):
                            image_splithalf1 = np.mean(neural_array[trial_indices[:25], image, :], axis = 0)
                            image_splithalf2 = np.mean(neural_array[trial_indices[25:], image, :], axis = 0)
                            image_pearson = pearsonr(np.nan_to_num(image_splithalf1), np.nan_to_num(image_splithalf2))[0]
                            iter_dict2 = {'iteration': i, 'instance': 'image' + str(image + 1),
                                          'splithalf_r': image_pearson, 'spearman_brown': spearman_brown(image_pearson)}
                            reliabilities_list.append({**iter_dict1, **iter_dict2})
        
    reliabilities = pd.DataFrame(reliabilities_list)
    reliabilities.to_csv(output_file)

In [ ]:
rdm_reliabilities = reliabilities[reliabilities['instance'].str.contains('rdm')]
image_reliabilities = reliabilities[reliabilities['instance'].str.contains('image')]

#### Reliability Selection Plotting

In [ ]:
sns.relplot(x='cutoff', y='splithalf_r', hue='neural_site', kind='line', 
            data=rdm_reliabilities.groupby(['cutoff','neural_site'])['splithalf_r'].mean().reset_index());

In [ ]:
sns.relplot(x='cutoff', y='splithalf_r', row='layer', col='area', hue='neural_site', kind='line', 
            data=rdm_reliabilities.groupby(['cutoff','area', 'layer', 'neural_site'])['splithalf_r'].mean().reset_index());

In [ ]:
sns.relplot(x='cutoff', y='splithalf_r', hue='neural_site', kind='line', 
            data=image_reliabilities.groupby(['cutoff','neural_site', 'instance'])['splithalf_r'].mean().reset_index());

In [ ]:
sns.relplot(x='cutoff', y='splithalf_r', row='layer', col='area', hue='neural_site', kind='line', 
            data=image_reliabilities.groupby(['cutoff','area', 'layer', 'neural_site', 'instance'])['splithalf_r'].mean().reset_index());

In [ ]:
reliable_neuron_count = 0
reliable_neurons = {}
for area in visual_areas:
    reliable_neurons[area] = {}
    for layer in neural_reliabilities[area].keys():
        reliable_neurons[area][layer] = []
        reliable_neurons_in_layer = 0
        for neural_index, neuron in enumerate(neural_reliabilities[area][layer]):
            if neuron[0] >= 0.75:
                reliable_neuron_count += 1
                reliable_neurons_in_layer += 1
                reliable_neurons[area][layer].append(neural_index)
        reliable_neurons[area][layer] = np.array(reliable_neurons[area][layer])
        print(area, layer, ':', reliable_neurons_in_layer)
print('total reliable neurons: ', reliable_neuron_count)

In [ ]:
neural_reliabilities_df.query('splithalf_r >= 0.75').groupby(['area','layer'])['neuron'].count()
print('total reliable neurons: ', neural_reliabilities_df.query('splithalf_r > 0.75')['neuron'].count())

In [ ]:
reliable_brain_rdms = {}
reliable_responses_byimage = {}
reliable_responses_bytrial = {}
for area in visual_areas:
    reliable_brain_rdms[area] = {}
    reliable_responses_byimage[area] = {}
    reliable_responses_bytrial[area] = {}
    for layer in neural_reliabilities[area].keys():
        reliable_responses_byimage[area][layer]=brain_responses_byimage[area][layer][:,reliable_neurons[area][layer]]
        reliable_responses_bytrial[area][layer]=brain_responses_bytrial[area][layer][:,reliable_neurons[area][layer]]
        reliable_brain_rdms[area][layer] = np.corrcoef(reliable_responses_byimage[area][layer])
        
output_file = 'rdms_reliable.pkl'
if not os.path.exists(output_file): 
    with open(output_file, 'wb') as file:
        pickle.dump(reliable_brain_rdms, file)

In [ ]:
from scipy.stats import pearsonr

target_responses = reliable_responses_bytrial

trial_indices = np.array(range(50))
neural_splithalf_rdms = {}
for area in tqdm(visual_areas):
    neural_splithalf_rdms[area] = {}
    layers = list(target_responses[area].keys())
    for layer in tqdm(layers, leave = False):
        neural_splithalf_rdms[area][layer] = {}
        response_array = target_responses[area][layer]
        neural_array = np.zeros(shape=(response_array.shape[1], response_array.shape[0] // 50, 50))
        for neuron in range(response_array.shape[1]):
                for image in range(response_array.shape[0] // 50):
                    neural_array[neuron][image] = response_array[50*image:50*(image+1), neuron]
                    
        neural_array = np.swapaxes(neural_array, 0, 2)
        for iteration in tqdm(range(10), desc = 'Similarity', leave = False):
            np.random.shuffle(trial_indices)
            similarity1 = np.corrcoef(np.mean(neural_array[trial_indices[:25], :, : ], axis=0))
            similarity2 = np.corrcoef(np.mean(neural_array[trial_indices[25:], :, : ], axis=0))
            sim1_triu = similarity1[np.triu_indices(similarity1.shape[0], k=1)].flatten()
            sim2_triu = similarity2[np.triu_indices(similarity2.shape[0], k=1)].flatten()
            sim_corr = pearsonr(np.nan_to_num(sim1_triu), np.nan_to_num(sim2_triu))[0]
            neural_splithalf_rdms[area][layer][iteration] = 2.0 * sim_corr / (1.0 + sim_corr)

In [ ]:
dictlist = []
for area in neural_splithalf_rdms.keys():
    for layer in neural_splithalf_rdms[area].keys():
        for iteration in neural_splithalf_rdms[area][layer].keys():
            dictlist.append({'area': area, 'layer': layer, 'similarity': neural_splithalf_rdms[area][layer][iteration], 
                             'id': 'neural_internal_' + str(iteration)})
            
neural_internal_rdm_summary = pd.DataFrame(dictlist)

output_file = 'rdms_splithalf.csv'
if not os.path.exists(output_file):
     neural_internal_rdm_summary.to_csv(output_file, index = None)

In [ ]:
sns.catplot(x='area', y='similarity', hue='layer', kind='bar', palette=sns.light_palette('navy'), 
            data=neural_internal_rdm_summary);

## Variance Calculations

In [ ]:
neural_stats_dictlist = []
for area in tqdm(visual_areas):
    layers = brain_responses_byimage[area].keys()
    for layer in tqdm(layers, leave = False):
        for neuron in range(brain_responses_byimage[area][layer].shape[1]):
            neural_mean = np.mean(brain_responses_byimage[area][layer][:,neuron])
            neural_sigma = np.var(brain_responses_byimage[area][layer][:,neuron])
            neural_stats_dictlist.append({'area': area, 'layer': layer, 'neural_site': area+'-'+layer,
                                          'neuron': neuron, 'mu': neural_mean, 'sigma': neural_sigma})
                
neural_stats = pd.DataFrame(neural_stats_dictlist)
output_file = 'summary_stats.csv'
if not os.path.exists(output_file):
    neural_stats.to_csv(output_file, index = None)

In [ ]:
neural_stats.groupby(['area', 'layer'])['mu'].describe()[['count','mean','std','min','max']].reset_index()

In [ ]:
neural_stats.groupby(['area', 'layer'])['sigma'].describe()[['count','mean','std','min','max']].reset_index()

In [ ]:
plt.hist(neural_stats['mu'], bins = 100)
plt.yscale('log')

In [ ]:
plt.hist(neural_stats['sigma'], bins = 100)
plt.yscale('log')

In [ ]:
from copy import copy
area = copy(visual_areas)
layers = ['layer23', 'layer4', 'layer5', 'layer6']
fig, axs = plt.subplots(4, 6, figsize=(16,8))
for i, area in enumerate(visual_areas):
    for j, layer in enumerate(layers):
        data = neural_stats[(neural_stats.area == area) & (neural_stats.layer == layer)]
        axs[j, i].hist(data['sigma'], bins = 100)
        axs[j, i].set_yscale('log')
        axs[j, i].set_title(area +'-'+layer)
fig.tight_layout()

### Variance Selection

In [ ]:
def pd_minmax_norm(df):
    return (df-df.min())/(df.max()-df.min())

neural_stats[['mu_norm', 'sigma_norm']] = pd_minmax_norm(neural_stats[['mu', 'sigma']])

In [ ]:
neural_stats['sigma'].mean(), neural_stats['sigma'].std()

In [ ]:
neural_stats['sigma_norm'].mean(), neural_stats['sigma_norm'].std()

In [ ]:
neural_stats[neural_stats['sigma_norm'] > .005]['neuron'].count()

In [ ]:
neural_stats[['mu_norm', 'sigma_norm']] = pd_minmax_norm(neural_stats[['mu', 'sigma']])
variance_count_list = []
for cutoff in tqdm(np.linspace(neural_stats['sigma_norm'].min(), .0001, 100)):
    for area in tqdm(visual_areas, leave = False):
        layers = brain_responses_byimage[area].keys()
        for layer in tqdm(layers, leave = False):
            neural_subset = neural_stats[(neural_stats['area'] == area) & (neural_stats['layer'] == layer)]
            neuron_count = neural_subset[neural_subset['sigma_norm'] > cutoff]['neuron'].count()
            variance_count_list.append({'area': area, 'layer': layer, 'neural_site': area+'-'+layer,
                                           'cutoff': cutoff, 'count': neuron_count})
variance_counts = pd.DataFrame(variance_count_list)

In [ ]:
sns.relplot(x='cutoff', y='count', hue='neural_site', data = variance_counts);

In [ ]:
sns.scatterplot(x='cutoff', y='count', data = variance_counts.groupby('cutoff')['count'].sum().reset_index());

In [ ]:
neural_stats[['mu_norm', 'sigma_norm']] = pd_minmax_norm(neural_stats[['mu', 'sigma']])
variance_count_list = []
for cutoff in tqdm(np.linspace(0,1,100)):
    for area in tqdm(visual_areas, leave = False):
        layers = brain_responses_byimage[area].keys()
        for layer in tqdm(layers, leave = False):
            neural_subset = neural_stats[(neural_stats['area'] == area) & (neural_stats['layer'] == layer)]
            #lower_bound = neural_subset['sigma_norm'].mean() - neural_subset['sigma_norm'].std()*cutoff
            #upper_bound = neural_subset['sigma_norm'].mean() + neural_subset['sigma_norm'].std()*cutoff
            #lower_bound_conditional = neural_subset['sigma_norm'] > lower_bound
            #upper_bound_conditional = neural_subset['sigma_norm'] < upper_bound
            #neuron_count = neural_subset[(lower_bound_conditional) & (upper_bound_conditional)]['neuron'].count()
            lower_bound = 0 + neural_subset['sigma_norm'].std()*cutoff
            neuron_count = neural_subset[neural_subset['sigma_norm'] > lower_bound]['neuron'].count()
            variance_count_list.append({'area': area, 'layer': layer, 'neural_site': area+'-'+layer,
                                           'cutoff': cutoff, 'count': neuron_count})
variance_counts = pd.DataFrame(variance_count_list)

In [ ]:
curve_data = variance_counts.groupby('cutoff')['count'].sum().reset_index()
xdata = curve_data['cutoff']
ydata = curve_data['count']